# Assessment of oxford government response dataset

This notebook assesses the oxford government response dataset.

## imports

In [19]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## define bucket

In [5]:
#FIXME
input_bucket = 's3://dh-ud-dend-capstone/stage0_raw'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Oxford data

In [6]:
#read in dataset
ox_file = '/oxford-gov-response/OxCGRT_latest.csv'
df_ox = spark.read.csv(input_bucket + ox_file,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# show a few records
df_ox.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+----------+----------+------------+--------+-----------------+-------+--------------------+-------+-----------------------+-------+-----------------------------+-------+-------------------------+-------+----------------------------+-------+------------------------------------+-------+--------------------------------+-----------------+-------+-----------------------+------------------+------------------------+-------------------------------+-------+-----------------+------------------+-------------------------------------+-------------------------+-------------------+-------+---------------------+-------+-----------+--------------+---------------+---------------+-------------------------+---------------------+-------------------------------+-----------------------+---------------------------------+----------------------+--------------------------------+--------------------+------------------------------+
|CountryName|CountryCode|RegionName|RegionCode|Jurisdictio

In [8]:
# count the records
df_ox.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

99680

In [9]:
# print schema
df_ox.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- CountryName: string (nullable = true)
 |-- CountryCode: string (nullable = true)
 |-- RegionName: string (nullable = true)
 |-- RegionCode: string (nullable = true)
 |-- Jurisdiction: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- C1_School closing: string (nullable = true)
 |-- C1_Flag: string (nullable = true)
 |-- C2_Workplace closing: string (nullable = true)
 |-- C2_Flag: string (nullable = true)
 |-- C3_Cancel public events: string (nullable = true)
 |-- C3_Flag: string (nullable = true)
 |-- C4_Restrictions on gatherings: string (nullable = true)
 |-- C4_Flag: string (nullable = true)
 |-- C5_Close public transport: string (nullable = true)
 |-- C5_Flag: string (nullable = true)
 |-- C6_Stay at home requirements: string (nullable = true)
 |-- C6_Flag: string (nullable = true)
 |-- C7_Restrictions on internal movement: string (nullable = true)
 |-- C7_Flag: string (nullable = true)
 |-- C8_International travel controls: string (nullable = true)
 |-- E

In [10]:
df_ox.createOrReplaceTempView("oxf_data")

#check most present keys
spark.sql("""select distinct CountryName,CountryCode,RegionName,RegionCode, Jurisdiction from oxf_data""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-----------+----------------+----------+------------+
|   CountryName|CountryCode|      RegionName|RegionCode|Jurisdiction|
+--------------+-----------+----------------+----------+------------+
|        Greece|        GRC|            null|      null|   NAT_TOTAL|
| United States|        USA|          Kansas|     US_KS| STATE_TOTAL|
| United States|        USA|         Alabama|     US_AL| STATE_TOTAL|
|         China|        CHN|            null|      null|   NAT_TOTAL|
|United Kingdom|        GBR|           Wales|    UK_WAL| STATE_TOTAL|
|        Brazil|        BRA|Distrito Federal|     BR_DF| STATE_TOTAL|
| United States|        USA|          Hawaii|     US_HI| STATE_TOTAL|
|       Estonia|        EST|            null|      null|   NAT_TOTAL|
|       Albania|        ALB|            null|      null|   NAT_TOTAL|
|          Oman|        OMN|            null|      null|   NAT_TOTAL|
|          Guam|        GUM|            null|      null|   NAT_TOTAL|
| United States|    

In [11]:
#check for records for which the country code is null
spark.sql("""select count(*) as cnt_cc_is_null from oxf_data where CountryCode is null""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|cnt_cc_is_null|
+--------------+
|             0|
+--------------+

In [12]:
#check for records for which Date is null
spark.sql("""select count(*) as cnt_date_is_null from oxf_data where Date is null""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+
|cnt_date_is_null|
+----------------+
|               0|
+----------------+

In [13]:
#select most present country codes
spark.sql("""select CountryCode, count(*) as c from oxf_data group by CountryCode order by c desc""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----+
|CountryCode|    c|
+-----------+-----+
|        USA|18868|
|        BRA| 9968|
|        CAN| 4984|
|        GBR| 1780|
|        LVA|  356|
|        BOL|  356|
|        BRB|  356|
|        URY|  356|
|        POL|  356|
|        JAM|  356|
|        ZMB|  356|
|        CUB|  356|
|        JOR|  356|
|        BRN|  356|
|        COD|  356|
|        ABW|  356|
|        FRA|  356|
|        SOM|  356|
|        ETH|  356|
|        MOZ|  356|
+-----------+-----+
only showing top 20 rows

In [14]:
#select number of dates and their minimum and maximum
spark.sql("""select count(distinct Date), min(Date), max(Date) from oxf_data""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+---------+
|count(DISTINCT Date)|min(Date)|max(Date)|
+--------------------+---------+---------+
|                 356| 20200101| 20201221|
+--------------------+---------+---------+

In [15]:
#check data format of ordinal metric columns
spark.sql("""select `C1_School closing`, count(*) from oxf_data group by `C1_School closing`""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+--------+
|C1_School closing|count(1)|
+-----------------+--------+
|                3|      19|
|             3.00|   41548|
|             null|    4574|
|             2.00|   17533|
|                1|     190|
|             1.00|   12954|
|             0.00|   22855|
|                2|       7|
+-----------------+--------+

In [17]:
#check content of column Jurisdiction
spark.sql("""select Jurisdiction, count(*) from oxf_data group by Jurisdiction""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------+
|Jurisdiction|count(1)|
+------------+--------+
|   NAT_TOTAL|   65504|
| STATE_TOTAL|   34176|
+------------+--------+

In [14]:
#show summary
df_ox.summary().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+-----------+----------+----------+------------+-------------------+------------------+------------------+--------------------+------------------+-----------------------+------------------+-----------------------------+-------------------+-------------------------+-------------------+----------------------------+-------------------+------------------------------------+------------------+--------------------------------+------------------+-------------------+-----------------------+--------------------+------------------------+-------------------------------+-------------------+------------------+------------------+-------------------------------------+-------------------------+-------------------+------------------+---------------------+-------+-----------+------------------+------------------+------------------+-------------------------+---------------------+-------------------------------+-----------------------+---------------------------------+------------------

In [21]:
# check if country code format is iso 3166
oxf_cc = df_ox.select('CountryName','CountryCode').distinct().orderBy('CountryCode')

input_bucket = 's3://dh-ud-dend-capstone/stage0_raw/google/'
df_gl_index = spark.read.format('csv').options(header='true').load(input_bucket + 'index.csv')

google_cc = df_gl_index.select('3166-1-alpha-2', '3166-1-alpha-3').distinct()

cc_joined = oxf_cc.alias('o').join(google_cc.alias('g'), on=[col('o.CountryCode') == col('g.3166-1-alpha-3')], how='left')

#count rows which couldn't match, should be 0
cc_joined.filter("`3166-1-alpha-3` is null").count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

### FINDINGS
* reduce columns
* check for duplicates
* check meaning of flags columns
* remove spaces from column names
* convert column types
* map Country Codes to ISO
* remove "ForDisplay" columns
* remove cases

* stage2: filter to jurisdiction NAT_TOTA = only national level